In [ ]:
# Important Libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading the dataset
file_path = r"C:\Users\tiles\Downloads\final_dataset 8.csv"
df = pd.read_csv(file_path)

In [ ]:
# Calculating target variables
df.loc[df['TS-Capacity']==0, 'TS-Capacity'] = 216  
df['perc_capacity'] = df['TS-Enrollment']/df['TS-Capacity']
df['net_rev_per_student'] = df['net_program_80']/df['TS-Enrollment']

In [ ]:
# Handling NA
fill_values = {
    'fulcrum_total_app': 0, 
    'fulcrum_grant_count': 0, 
    'fulcrum_grant_mean': 0,
    'Nonfamily Avg Household Size': df.groupby('Year')['Nonfamily Avg Household Size'].transform('mean'), 
    'Med inc_Med income families': df['Med inc_all households']
}
# Checking filling missing values 
df = df.fillna(value=fill_values)

In [ ]:
# Columns to drop which can affect the overall model development 
cols_to_drop = [
    'Unnamed: 0', 'school_id', 'Name', 'zip_code', 'Type', 'type', 'level', 'District', 'School',
    'TS-Catholic', 'TS-Non-Catholic', 'Income At or Above Poverty Level', 
    'TPS-Non-Catholic', 'TPS-Catholic', 'Female (Female)', 'Male (Male)',
    'child_under_5', 'child_5_9', 'child_10_14', 
    'pub_enroll_all_students',
    'PercentLevel1_ELA', 'PercentLevel1_Math', 'PercentLevel2_ELA', 'PercentLevel2_Math',
    'PercentLevel3_ELA', 'PercentLevel3_Math', 'PercentLevel4_ELA', 'PercentLevel4_Math',
    'parish_ID', 'parish_name&city', 'short_name', 'Year_x', 'Parish', 'TS-Capacity', 'TS-Enrollment'
]

In [ ]:
# Checking the current data
df_filtered = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

print(f"Original shape: {df.shape}")
print(f"Filtered shape: {df_filtered.shape}")
print(f"Removed {len([col for col in cols_to_drop if col in df.columns])} columns")

#### Variable Importance 

##### T - Test

In [ ]:
# Perform t-test
def perform_t_test(df, features, target, threshold=0.05):
    t_test_results = {}
    
    # Median of target variable to split data
    median_target = df[target].median()
    group_high = df[df[target] > median_target]
    group_low = df[df[target] <= median_target]
    
    significant_features = []
    
    for feature in features:
        if df[feature].isna().sum() > 0:
            continue
        if df[feature].dtype not in [np.float64, np.int64]:
            continue
            
        values_high = group_high[feature].values
        values_low = group_low[feature].values
        
        # Performing t-test
        t_stat, p_value = stats.ttest_ind(values_high, values_low, equal_var=False)
        
        t_test_results[feature] = {
            't_statistic': t_stat,
            'p_value': p_value,
            'significant': p_value < threshold
        }
        
        if p_value < threshold:
            significant_features.append(feature)
    
    return t_test_results, significant_features


In [ ]:
# Getting numeric features 
numeric_features = df_filtered.select_dtypes(include=[np.float64, np.int64]).columns.tolist()
numeric_features = [col for col in numeric_features if col not in ['perc_capacity', 'net_rev_per_student']]

print(f"\nNumber of filtered numeric features: {len(numeric_features)}")

# Performing t-test for both target variables
print("\nPerforming T-test for perc_capacity...")
t_test_perc_capacity, sig_features_perc_capacity = perform_t_test(df_filtered, numeric_features, 'perc_capacity')

print("\nPerforming T-test for net_rev_per_student...")
t_test_net_rev, sig_features_net_rev = perform_t_test(df_filtered, numeric_features, 'net_rev_per_student')

# Significance of features
print(f"\nSignificant features for perc_capacity: {len(sig_features_perc_capacity)}")
sig_perc_capacity_df = pd.DataFrame({
    'Feature': list(t_test_perc_capacity.keys()), 
    'P-value': [t_test_perc_capacity[f]['p_value'] for f in t_test_perc_capacity],
    'Significant': [t_test_perc_capacity[f]['significant'] for f in t_test_perc_capacity]
})
sig_perc_capacity_df = sig_perc_capacity_df.sort_values('P-value')
print(sig_perc_capacity_df[sig_perc_capacity_df['Significant'] == True].head(15))

print(f"\nSignificant features for net_rev_per_student: {len(sig_features_net_rev)}")
sig_net_rev_df = pd.DataFrame({
    'Feature': list(t_test_net_rev.keys()), 
    'P-value': [t_test_net_rev[f]['p_value'] for f in t_test_net_rev],
    'Significant': [t_test_net_rev[f]['significant'] for f in t_test_net_rev]
})
sig_net_rev_df = sig_net_rev_df.sort_values('P-value')
print(sig_net_rev_df[sig_net_rev_df['Significant'] == True].head(15))

# Common Significant features
common_sig_features = list(set(sig_features_perc_capacity) & set(sig_features_net_rev))
print(f"\nCommon significant features between both targets: {len(common_sig_features)}")
print(common_sig_features)

#### Spearman Correlation

In [ ]:
# Function to calculate Spearman correlation with target variable
def perform_spearman_correlation(df, features, target):
    corr_results = {}
    
    for feature in features:
        if df[feature].isna().sum() == 0:  # Skip features with missing values
            correlation, p_value = stats.spearmanr(df[feature], df[target])
            corr_results[feature] = {
                'correlation': correlation,
                'p_value': p_value,
                'significant': p_value < 0.05
            }
    
    # Convert to DataFrame
    corr_df = pd.DataFrame({
        'Feature': list(corr_results.keys()),
        'Correlation': [corr_results[f]['correlation'] for f in corr_results],
        'P-value': [corr_results[f]['p_value'] for f in corr_results],
        'Significant': [corr_results[f]['significant'] for f in corr_results]
    })
    
    # Sort by absolute correlation
    corr_df['Abs_Correlation'] = corr_df['Correlation'].abs()
    corr_df = corr_df.sort_values('Abs_Correlation', ascending=False)
    
    return corr_df

In [ ]:
# Numeric features 
numeric_features = df_filtered.select_dtypes(include=[np.float64, np.int64]).columns.tolist()
numeric_features = [col for col in numeric_features if col not in ['perc_capacity', 'net_rev_per_student']]

print(f"Number of filtered numeric features for Spearman correlation: {len(numeric_features)}")

In [ ]:
# Spearman Correlation Calculation
# Calculate Spearman correlation for perc_capacity
print("\nCalculating Spearman correlation for perc_capacity...")
corr_perc_capacity = perform_spearman_correlation(df_filtered, numeric_features, 'perc_capacity')

# Calculate Spearman correlation for net_rev_per_student
print("\nCalculating Spearman correlation for net_rev_per_student...")
corr_net_rev = perform_spearman_correlation(df_filtered, numeric_features, 'net_rev_per_student')

In [ ]:
# Significant features with strongest correlations
print("\nTop significant features by Spearman correlation for perc_capacity:")
sig_perc_corr = corr_perc_capacity[corr_perc_capacity['Significant'] == True]
print(sig_perc_corr[['Feature', 'Correlation', 'P-value']].head(15))

print("\nTop significant features by Spearman correlation for net_rev_per_student:")
sig_net_rev_corr = corr_net_rev[corr_net_rev['Significant'] == True]
print(sig_net_rev_corr[['Feature', 'Correlation', 'P-value']].head(15))

In [ ]:
# Features with significant correlation for both targets
sig_perc_features = set(sig_perc_corr['Feature'].tolist())
sig_net_rev_features = set(sig_net_rev_corr['Feature'].tolist())
common_sig_corr_features = list(sig_perc_features.intersection(sig_net_rev_features))

print(f"\nFeatures with significant Spearman correlation for both targets: {len(common_sig_corr_features)}")
print(common_sig_corr_features)

In [ ]:
# Random Forest Permutation importance analysis
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Get numeric features from filtered dataset
numeric_features = df_filtered.select_dtypes(include=[np.float64, np.int64]).columns.tolist()
numeric_features = [col for col in numeric_features if col not in ['perc_capacity', 'net_rev_per_student']]

In [ ]:
# Prepare data for permutation importance
X = df_filtered[numeric_features].copy()
X = X.fillna(X.mean())  # Fill missing values with mean

# Get target variables
y_perc_capacity = df_filtered['perc_capacity']
y_net_rev = df_filtered['net_rev_per_student']

print("Prepared data for Random Forest permutation importance analysis")
print(f"Number of features being evaluated: {X.shape[1]}")

In [ ]:
# Function to calculate permutation importance
def get_permutation_importance(X, y, feature_names):
    # Split the data - we need a test set for permutation importance
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Train Random Forest model
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    # Calculate R2 and RMSE on test set (informational only)
    y_pred = rf.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Calculate permutation importance on test set
    # This shows how much model performance decreases when a feature is randomly shuffled
    perm_importance = permutation_importance(rf, X_test, y_test, 
                                            n_repeats=5,  # Fewer repeats to speed up calculation
                                            random_state=42,
                                            n_jobs=-1)
    
    # Create DataFrame with results
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': perm_importance.importances_mean,
        'Std': perm_importance.importances_std
    }).sort_values('Importance', ascending=False)
    
    return importance_df, r2, rmse

In [ ]:
# Calculate permutation importance for perc_capacity
print("\nCalculating permutation importance for perc_capacity...")
perm_imp_perc, r2_perc, rmse_perc = get_permutation_importance(X, y_perc_capacity, numeric_features)

# Calculate permutation importance for net_rev_per_student
print("Calculating permutation importance for net_rev_per_student...")
perm_imp_net, r2_net, rmse_net = get_permutation_importance(X, y_net_rev, numeric_features)

In [ ]:
# Display results for perc_capacity
print("\nPermutation importance for perc_capacity:")
print(f"Model R² Score: {r2_perc:.4f}, RMSE: {rmse_perc:.4f}")
print("\nTop 15 features by permutation importance:")
print(perm_imp_perc.head(15))

# Display results for net_rev_per_student
print("\nPermutation importance for net_rev_per_student:")
print(f"Model R² Score: {r2_net:.4f}, RMSE: {rmse_net:.4f}")
print("\nTop 15 features by permutation importance:")
print(perm_imp_net.head(15))

In [ ]:
# Save results to variables with clear names for later meta-analysis
rf_perm_imp_features_perc_capacity = perm_imp_perc.head(15)['Feature'].tolist()
rf_perm_imp_features_net_rev = perm_imp_net.head(15)['Feature'].tolist()

In [ ]:
# Stability Selection for Feature Importance
from sklearn.linear_model import Lasso, LassoCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Get numeric features from filtered dataset
numeric_features = df_filtered.select_dtypes(include=[np.float64, np.int64]).columns.tolist()
numeric_features = [col for col in numeric_features if col not in ['perc_capacity', 'net_rev_per_student']]


In [ ]:
# Prepare data
X = df_filtered[numeric_features].copy()
X = X.fillna(X.mean())  # Fill missing values with mean

# Get target variables
y_perc_capacity = df_filtered['perc_capacity']
y_net_rev = df_filtered['net_rev_per_student']

print("Prepared data for Stability Selection")
print(f"Number of features being evaluated: {X.shape[1]}")

In [ ]:
# Function to perform stability selection
def stability_selection(X, y, feature_names, n_subsamples=100, sample_fraction=0.75, threshold=0.5):
    n_samples, n_features = X.shape
    subsample_size = int(n_samples * sample_fraction)
    
    # Track feature selection frequency
    selection_frequency = np.zeros(n_features)
    
    for i in range(n_subsamples):
        # Random subsample
        subsample_indices = np.random.choice(n_samples, subsample_size, replace=False)
        X_subsample = X.iloc[subsample_indices]
        y_subsample = y.iloc[subsample_indices]
        
        # Find optimal alpha with cross-validation
        lasso_cv = LassoCV(cv=5, random_state=i)
        lasso_cv.fit(X_subsample, y_subsample)
        
        # Fit Lasso with selected alpha
        lasso = Lasso(alpha=lasso_cv.alpha_)
        lasso.fit(X_subsample, y_subsample)
        
        # Track which features were selected (non-zero coefficients)
        selection_frequency += (lasso.coef_ != 0).astype(int)
    
    # Calculate selection probability for each feature
    selection_probability = selection_frequency / n_subsamples
    
    # Create results DataFrame
    results = pd.DataFrame({
        'Feature': feature_names,
        'Selection_Probability': selection_probability
    })
    
    # Sort by selection probability
    results = results.sort_values('Selection_Probability', ascending=False)
    
    # Get stable features based on threshold
    stable_features = results[results['Selection_Probability'] >= threshold]['Feature'].tolist()
    
    return results, stable_features


In [ ]:
# Perform stability selection for perc_capacity
print("\nPerforming stability selection for perc_capacity...")
stability_results_perc, stable_features_perc = stability_selection(
    X, y_perc_capacity, numeric_features, n_subsamples=50  # Reduced for speed
)

# Perform stability selection for net_rev_per_student
print("Performing stability selection for net_rev_per_student...")
stability_results_net, stable_features_net = stability_selection(
    X, y_net_rev, numeric_features, n_subsamples=50  # Reduced for speed
)

# Show results
print("\nStability selection results for perc_capacity:")
print(f"Number of stable features: {len(stable_features_perc)}")
print(stability_results_perc.head(15))

print("\nStability selection results for net_rev_per_student:")
print(f"Number of stable features: {len(stable_features_net)}")
print(stability_results_net.head(15))

In [ ]:
# Meta Analysis of Feature Selection Methods
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 1. From T-test
t_test_features_perc = sig_perc_capacity_df[sig_perc_capacity_df['Significant'] == True]['Feature'].tolist()
print(f"\nT-test significant features for perc_capacity: {len(t_test_features_perc)}")

# 2. From Spearman Correlation
spearman_features_perc = sig_perc_corr['Feature'].tolist()
print(f"Spearman significant features for perc_capacity: {len(spearman_features_perc)}")

# 3. From Random Forest Permutation Importance (top 15)
rf_features_perc = perm_imp_perc.head(15)['Feature'].tolist()
print(f"Random Forest important features for perc_capacity: {len(rf_features_perc)}")

# 4. From Stability Selection
stability_features_perc = stability_results_perc[stability_results_perc['Selection_Probability'] >= 0.5]['Feature'].tolist()
print(f"Stability Selection stable features for perc_capacity: {len(stability_features_perc)}")

In [ ]:
# For net_rev_per_student target:
# 1. From T-test
t_test_features_net = sig_net_rev_df[sig_net_rev_df['Significant'] == True]['Feature'].tolist()
print(f"\nT-test significant features for net_rev_per_student: {len(t_test_features_net)}")

# 2. From Spearman Correlation
spearman_features_net = sig_net_rev_corr['Feature'].tolist()
print(f"Spearman significant features for net_rev_per_student: {len(spearman_features_net)}")

# 3. From Random Forest Permutation Importance (top 15)
rf_features_net = perm_imp_net.head(15)['Feature'].tolist()
print(f"Random Forest important features for net_rev_per_student: {len(rf_features_net)}")

# 4. From Stability Selection
stability_features_net = stability_results_net[stability_results_net['Selection_Probability'] >= 0.5]['Feature'].tolist()
print(f"Stability Selection stable features for net_rev_per_student: {len(stability_features_net)}")


In [ ]:
# Create a feature scoring system for each target
# For each feature, count how many methods selected it
feature_counts_perc = {}
feature_counts_net = {}

# Get all possible features
all_features = set(t_test_features_perc + spearman_features_perc + rf_features_perc + stability_features_perc + 
                  t_test_features_net + spearman_features_net + rf_features_net + stability_features_net)

# Count occurrences for each feature for perc_capacity
for feature in all_features:
    # Count for perc_capacity
    count_perc = 0
    if feature in t_test_features_perc:
        count_perc += 1
    if feature in spearman_features_perc:
        count_perc += 1
    if feature in rf_features_perc:
        count_perc += 1
    if feature in stability_features_perc:
        count_perc += 1
    
    feature_counts_perc[feature] = count_perc
    
    # Count for net_rev_per_student
    count_net = 0
    if feature in t_test_features_net:
        count_net += 1
    if feature in spearman_features_net:
        count_net += 1
    if feature in rf_features_net:
        count_net += 1
    if feature in stability_features_net:
        count_net += 1
    
    feature_counts_net[feature] = count_net

In [ ]:
# Convert to DataFrames
feature_scores_perc = pd.DataFrame({
    'Feature': list(feature_counts_perc.keys()),
    'Selection_Count': list(feature_counts_perc.values())
}).sort_values('Selection_Count', ascending=False)

feature_scores_net = pd.DataFrame({
    'Feature': list(feature_counts_net.keys()),
    'Selection_Count': list(feature_counts_net.values())
}).sort_values('Selection_Count', ascending=False)

# Display features selected by multiple methods
print("\nTop features for perc_capacity (selected by multiple methods):")
print(feature_scores_perc[feature_scores_perc['Selection_Count'] >= 2].head(15))

print("\nTop features for net_rev_per_student (selected by multiple methods):")
print(feature_scores_net[feature_scores_net['Selection_Count'] >= 2].head(15))

In [ ]:
# Select features that appeared in at least 2 methods
selected_features_perc = feature_scores_perc[feature_scores_perc['Selection_Count'] >= 2]['Feature'].tolist()
selected_features_net = feature_scores_net[feature_scores_net['Selection_Count'] >= 2]['Feature'].tolist()

print(f"\nSelected features for perc_capacity: {len(selected_features_perc)}")
print(selected_features_perc)

print(f"\nSelected features for net_rev_per_student: {len(selected_features_net)}")
print(selected_features_net)

#### Linear Discriminant Analysis Classification

In [ ]:
# Linear Discriminant Analysis Classification
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Define selected features for each target variable
selected_features_perc = ['tuition_fees_finaid_scholarships', 'benefits_52xx', 'Med inc_all households', 'net_program_80', 'salaries_51xx', 'total_expenses_80', 'program_expenses', 'supplies', 'total_revenue_80', 'all_other_expenses', 'T-Parish', 'repairs_maintenance_58xx', 'female_young_adults', 'all_other_revenue', 'contracted_services', 'fundraising_parents_club', 'utilities', 'TPS-Employment', 'bequests', 'PercentMetStandard_ELA', 'Med inc_Med income families', 'business_revenue_45xx', 'PercentMetStandard_Math', 'Nonfamily Total Households', 'Nonfamily Avg Household Size', 'male_young_adults']

selected_features_net = ['net_program_80', 'all_other_revenue', 'Med inc_Med income families', 'TPS-Employment', 'T-Parish', 'tuition_fees_finaid_scholarships', 'Owner-Occupied Units', 'total_expenses_80', 'Married Avg Family Size', 'TS_Non_catholic_ratio', 'Married Avg Household Size', 'Avg Household Size', 'total_revenue_80', 'Avg Family Size', 'gift_revenue_44xx', 'fundraising_parents_club']

In [ ]:
# Prepare data for classification
# Convert continuous targets to categorical (binary for simplicity)
# Calculate median of target variables to split into binary classes
perc_capacity_median = df_filtered['perc_capacity'].median()
net_rev_median = df_filtered['net_rev_per_student'].median()

# Create binary target variables
df_filtered['perc_capacity_binary'] = (df_filtered['perc_capacity'] > perc_capacity_median).astype(int)
df_filtered['net_rev_binary'] = (df_filtered['net_rev_per_student'] > net_rev_median).astype(int)

print(f"\nClass distribution for perc_capacity_binary:")
print(df_filtered['perc_capacity_binary'].value_counts())

print(f"\nClass distribution for net_rev_binary:")
print(df_filtered['net_rev_binary'].value_counts())

# Function to build and evaluate LDA model
def build_lda_model(X, y, feature_names):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Create a pipeline with scaling and LDA
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('lda', LinearDiscriminantAnalysis())
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Predictions
    y_pred = pipeline.predict(X_test)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    
    # Get LDA coefficients for feature importance
    lda = pipeline.named_steps['lda']
    scaler = pipeline.named_steps['scaler']
    
    # Get coefficient importance (use absolute values)
    coef = np.abs(lda.coef_[0])
    
    # Scale coefficients by the scaler to get feature importance
    scaled_coef = coef * scaler.scale_
    
    # Create feature importance dataframe
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': scaled_coef
    }).sort_values('Importance', ascending=False)
    
    return {
        'pipeline': pipeline,
        'accuracy': accuracy,
        'class_report': class_report,
        'conf_matrix': conf_matrix,
        'cv_scores': cv_scores,
        'feature_importance': feature_importance
    }

# Prepare data for perc_capacity model using selected features
print("\n==== Building LDA model for perc_capacity ====")
# Filter features that exist in the dataset
valid_perc_features = [f for f in selected_features_perc if f in df_filtered.columns]
print(f"Valid features for perc_capacity: {len(valid_perc_features)}/{len(selected_features_perc)}")

X_perc = df_filtered[valid_perc_features].copy()
X_perc = X_perc.fillna(X_perc.mean())  # Fill missing values
y_perc = df_filtered['perc_capacity_binary']

# Build LDA model for perc_capacity
lda_perc = build_lda_model(X_perc, y_perc, valid_perc_features)

# Prepare data for net_rev model using selected features
print("\n==== Building LDA model for net_rev_per_student ====")
# Filter features that exist in the dataset
valid_net_features = [f for f in selected_features_net if f in df_filtered.columns]
print(f"Valid features for net_rev_per_student: {len(valid_net_features)}/{len(selected_features_net)}")

X_net = df_filtered[valid_net_features].copy()
X_net = X_net.fillna(X_net.mean())  # Fill missing values
y_net = df_filtered['net_rev_binary']

# Build LDA model for net_rev_per_student
lda_net = build_lda_model(X_net, y_net, valid_net_features)

# Display results
print("\n==== LDA Results for perc_capacity ====")
print(f"Test Accuracy: {lda_perc['accuracy']:.4f}")
print(f"Cross-Validation Accuracy: {lda_perc['cv_scores'].mean():.4f} ± {lda_perc['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(lda_perc['class_report'])
print("\nTop 10 Important Features:")
print(lda_perc['feature_importance'].head(10))

print("\n==== LDA Results for net_rev_per_student ====")
print(f"Test Accuracy: {lda_net['accuracy']:.4f}")
print(f"Cross-Validation Accuracy: {lda_net['cv_scores'].mean():.4f} ± {lda_net['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(lda_net['class_report'])
print("\nTop 10 Important Features:")
print(lda_net['feature_importance'].head(10))

# Visualize LDA results
plt.figure(figsize=(20, 10))

# Plot confusion matrices
plt.subplot(2, 2, 1)
sns.heatmap(lda_perc['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - perc_capacity')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(2, 2, 2)
sns.heatmap(lda_net['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - net_rev_per_student')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Plot feature importance
plt.subplot(2, 2, 3)
top10_perc = lda_perc['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_perc)
plt.title('Top 10 Features - perc_capacity LDA')
plt.xlabel('Importance')

plt.subplot(2, 2, 4)
top10_net = lda_net['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_net)
plt.title('Top 10 Features - net_rev_per_student LDA')
plt.xlabel('Importance')

plt.tight_layout()
plt.savefig('lda_results_separate.png')
plt.show()

# Compare model performance across targets
models = ['perc_capacity', 'net_rev_per_student']
accuracies = [
    lda_perc['accuracy'], 
    lda_net['accuracy']
]
cv_accuracies = [
    lda_perc['cv_scores'].mean(),
    lda_net['cv_scores'].mean()
]

plt.figure(figsize=(10, 6))
x = np.arange(len(models))
width = 0.35

plt.bar(x - width/2, accuracies, width, label='Test Accuracy')
plt.bar(x + width/2, cv_accuracies, width, label='CV Accuracy')

plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('LDA Model Performance Comparison')
plt.xticks(x, models)
plt.legend()
plt.tight_layout()
plt.savefig('lda_performance_comparison_separate.png')
plt.show()

print("\nLDA classification analysis completed with separate feature sets!")

In [ ]:
# Random Forest and SVM Classification Models - Separate Features Only
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Note: We're using the feature lists defined in the previous code
# No need to redefine selected_features_perc and selected_features_net

# Function to build and evaluate classification models
def build_classification_model(X, y, feature_names, model_type='rf'):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Create a pipeline with scaling and the classifier
    if model_type == 'rf':
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
        ])
    elif model_type == 'svm':
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', SVC(probability=True, random_state=42))
        ])
    else:
        raise ValueError("model_type must be 'rf' or 'svm'")
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Predictions
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    
    # Get feature importance (for Random Forest only)
    if model_type == 'rf':
        rf = pipeline.named_steps['classifier']
        feature_importance = pd.DataFrame({
            'Feature': feature_names,
            'Importance': rf.feature_importances_
        }).sort_values('Importance', ascending=False)
    else:
        feature_importance = None
    
    return {
        'pipeline': pipeline,
        'accuracy': accuracy,
        'class_report': class_report,
        'conf_matrix': conf_matrix,
        'cv_scores': cv_scores,
        'feature_importance': feature_importance,
        'roc_auc': roc_auc,
        'fpr': fpr,
        'tpr': tpr
    }

# Prepare data for models - Filter features that exist in the dataset
valid_perc_features = [f for f in selected_features_perc if f in df_filtered.columns]
valid_net_features = [f for f in selected_features_net if f in df_filtered.columns]

print(f"\nValid features for perc_capacity: {len(valid_perc_features)}/{len(selected_features_perc)}")
print(f"Valid features for net_rev_per_student: {len(valid_net_features)}/{len(selected_features_net)}")

# Prepare data for perc_capacity model
X_perc = df_filtered[valid_perc_features].copy()
X_perc = X_perc.fillna(X_perc.mean())  # Fill missing values
y_perc = df_filtered['perc_capacity_binary']

# Prepare data for net_rev model
X_net = df_filtered[valid_net_features].copy()
X_net = X_net.fillna(X_net.mean())  # Fill missing values
y_net = df_filtered['net_rev_binary']

# Build Random Forest models
print("\n==== Building Random Forest models ====")
rf_perc = build_classification_model(X_perc, y_perc, valid_perc_features, model_type='rf')
rf_net = build_classification_model(X_net, y_net, valid_net_features, model_type='rf')

# Build SVM models
print("\n==== Building SVM models ====")
svm_perc = build_classification_model(X_perc, y_perc, valid_perc_features, model_type='svm')
svm_net = build_classification_model(X_net, y_net, valid_net_features, model_type='svm')

# Display Random Forest results
print("\n==== Random Forest Results for perc_capacity ====")
print(f"Test Accuracy: {rf_perc['accuracy']:.4f}")
print(f"ROC AUC: {rf_perc['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {rf_perc['cv_scores'].mean():.4f} ± {rf_perc['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(rf_perc['class_report'])
print("\nTop 10 Important Features:")
print(rf_perc['feature_importance'].head(10))

print("\n==== Random Forest Results for net_rev_per_student ====")
print(f"Test Accuracy: {rf_net['accuracy']:.4f}")
print(f"ROC AUC: {rf_net['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {rf_net['cv_scores'].mean():.4f} ± {rf_net['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(rf_net['class_report'])
print("\nTop 10 Important Features:")
print(rf_net['feature_importance'].head(10))

# Display SVM results
print("\n==== SVM Results for perc_capacity ====")
print(f"Test Accuracy: {svm_perc['accuracy']:.4f}")
print(f"ROC AUC: {svm_perc['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {svm_perc['cv_scores'].mean():.4f} ± {svm_perc['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(svm_perc['class_report'])

print("\n==== SVM Results for net_rev_per_student ====")
print(f"Test Accuracy: {svm_net['accuracy']:.4f}")
print(f"ROC AUC: {svm_net['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {svm_net['cv_scores'].mean():.4f} ± {svm_net['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(svm_net['class_report'])

# Visualize results
plt.figure(figsize=(20, 15))

# Plot confusion matrices for Random Forest
plt.subplot(2, 2, 1)
sns.heatmap(rf_perc['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('RF Confusion Matrix - perc_capacity')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(2, 2, 2)
sns.heatmap(rf_net['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('RF Confusion Matrix - net_rev_per_student')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Plot ROC curves
plt.subplot(2, 2, 3)
plt.plot(rf_perc['fpr'], rf_perc['tpr'], label=f'RF (AUC = {rf_perc["roc_auc"]:.3f})')
plt.plot(svm_perc['fpr'], svm_perc['tpr'], label=f'SVM (AUC = {svm_perc["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - perc_capacity')
plt.legend(loc='lower right')

plt.subplot(2, 2, 4)
plt.plot(rf_net['fpr'], rf_net['tpr'], label=f'RF (AUC = {rf_net["roc_auc"]:.3f})')
plt.plot(svm_net['fpr'], svm_net['tpr'], label=f'SVM (AUC = {svm_net["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - net_rev_per_student')
plt.legend(loc='lower right')

plt.tight_layout()
plt.savefig('rf_svm_results_separate.png')
plt.show()

# Feature importance visualization for Random Forest
plt.figure(figsize=(14, 10))

# For perc_capacity
plt.subplot(1, 2, 1)
top10_perc = rf_perc['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_perc)
plt.title('RF Top 10 Features - perc_capacity')
plt.xlabel('Importance')

# For net_rev_per_student
plt.subplot(1, 2, 2)
top10_net = rf_net['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_net)
plt.title('RF Top 10 Features - net_rev_per_student')
plt.xlabel('Importance')

plt.tight_layout()
plt.savefig('rf_feature_importance_separate.png')
plt.show()

# Compare model performance across methods and targets
models = ['RF-perc', 'RF-net', 'SVM-perc', 'SVM-net']
accuracies = [rf_perc['accuracy'], rf_net['accuracy'], svm_perc['accuracy'], svm_net['accuracy']]
cv_accuracies = [rf_perc['cv_scores'].mean(), rf_net['cv_scores'].mean(), svm_perc['cv_scores'].mean(), svm_net['cv_scores'].mean()]

plt.figure(figsize=(12, 6))
x = np.arange(len(models))
width = 0.35

plt.bar(x - width/2, accuracies, width, label='Test Accuracy')
plt.bar(x + width/2, cv_accuracies, width, label='CV Accuracy')

plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Model Performance Comparison')
plt.xticks(x, models)
plt.legend()
plt.tight_layout()
plt.savefig('model_comparison_separate.png')
plt.show()

print("\nRandom Forest and SVM classification analysis completed!")

In [ ]:
# Linear Discriminant Analysis Classification Script
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Prepare binary target variables
# Calculate median of target variables to split into binary classes
perc_capacity_median = df_filtered['perc_capacity'].median()
net_rev_median = df_filtered['net_rev_per_student'].median()

# Create binary target variables
df_filtered['perc_capacity_binary'] = (df_filtered['perc_capacity'] > perc_capacity_median).astype(int)
df_filtered['net_rev_binary'] = (df_filtered['net_rev_per_student'] > net_rev_median).astype(int)
# Create deficit binary target variable
# Create a 3-category financial status variable
def create_financial_status(net_program):
    if net_program < -10000:
        return 1  # Deficit
    elif net_program > 10000:
        return 2  # Surplus
    else:
        return 0  # Balanced

# Apply the function to create the financial status variable
df_filtered['financial_status'] = df_filtered['net_program_80'].apply(create_financial_status)

# For backward compatibility, also update the deficit_binary to use the -10000 threshold
df_filtered['deficit_binary'] = (df_filtered['net_program_80'] < -10000).astype(int)

# Print the distributions of both variables
print("\nUpdated deficit_binary distribution:")
print(df_filtered['deficit_binary'].value_counts())
print(f"Percentage of schools with significant deficit (< -$10,000): {df_filtered['deficit_binary'].mean() * 100:.2f}%")

print("\nFinancial status distribution:")
status_counts = df_filtered['financial_status'].value_counts()
status_percent = df_filtered['financial_status'].value_counts(normalize=True) * 100

print("Counts:")
for status, count in status_counts.items():
    if status == 0:
        status_name = "Balanced (-$10K to $10K)"
    elif status == 1:
        status_name = "Deficit (< -$10K)"
    else:
        status_name = "Surplus (> $10K)"
    print(f"{status_name}: {count} ({status_percent[status]:.2f}%)")
print("Class distribution for binary target variables:")
print("\nperc_capacity_binary:")
print(df_filtered['perc_capacity_binary'].value_counts())

print("\nnet_rev_binary:")
print(df_filtered['net_rev_binary'].value_counts())

print("\ndeficit_binary:")
print(df_filtered['deficit_binary'].value_counts())
print(f"Percentage of schools with deficit: {df_filtered['deficit_binary'].mean() * 100:.2f}%")

# Define features for deficit prediction - exclude net_program_80 which is used to calculate the target
deficit_features = [f for f in selected_features_net if f != 'net_program_80']

# Function to build and evaluate LDA model with ROC AUC calculation
def build_lda_model(X, y, feature_names):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Create a pipeline with scaling and LDA
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('lda', LinearDiscriminantAnalysis())
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Predictions
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    
    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    
    # Get LDA coefficients for feature importance
    lda = pipeline.named_steps['lda']
    scaler = pipeline.named_steps['scaler']
    
    # Get coefficient importance (use absolute values)
    coef = np.abs(lda.coef_[0])
    
    # Scale coefficients by the scaler to get feature importance
    scaled_coef = coef * scaler.scale_
    
    # Create feature importance dataframe
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': scaled_coef
    }).sort_values('Importance', ascending=False)
    
    return {
        'pipeline': pipeline,
        'accuracy': accuracy,
        'class_report': class_report,
        'conf_matrix': conf_matrix,
        'cv_scores': cv_scores,
        'feature_importance': feature_importance,
        'roc_auc': roc_auc,
        'fpr': fpr,
        'tpr': tpr
    }

# Prepare data for models - filter features that exist in the dataset
valid_perc_features = [f for f in selected_features_perc if f in df_filtered.columns]
valid_net_features = [f for f in selected_features_net if f in df_filtered.columns]
valid_deficit_features = [f for f in deficit_features if f in df_filtered.columns]

print(f"\nValid features for perc_capacity: {len(valid_perc_features)}/{len(selected_features_perc)}")
print(f"Valid features for net_rev_per_student: {len(valid_net_features)}/{len(selected_features_net)}")
print(f"Valid features for deficit prediction: {len(valid_deficit_features)}/{len(deficit_features)}")

# Prepare data for perc_capacity model
X_perc = df_filtered[valid_perc_features].copy()
X_perc = X_perc.fillna(X_perc.mean())  # Fill missing values
y_perc = df_filtered['perc_capacity_binary']

# Prepare data for net_rev model
X_net = df_filtered[valid_net_features].copy()
X_net = X_net.fillna(X_net.mean())  # Fill missing values
y_net = df_filtered['net_rev_binary']

# Prepare data for deficit model
X_deficit = df_filtered[valid_deficit_features].copy()
X_deficit = X_deficit.fillna(X_deficit.mean())  # Fill missing values
y_deficit = df_filtered['deficit_binary']

# Build LDA models
print("\n==== Building LDA models ====")
lda_perc = build_lda_model(X_perc, y_perc, valid_perc_features)
lda_net = build_lda_model(X_net, y_net, valid_net_features)
lda_deficit = build_lda_model(X_deficit, y_deficit, valid_deficit_features)

# Display results for perc_capacity
print("\n==== LDA Results for perc_capacity ====")
print(f"Test Accuracy: {lda_perc['accuracy']:.4f}")
print(f"ROC AUC: {lda_perc['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {lda_perc['cv_scores'].mean():.4f} ± {lda_perc['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(lda_perc['class_report'])
print("\nTop 10 Important Features:")
print(lda_perc['feature_importance'].head(10))

# Display results for net_rev
print("\n==== LDA Results for net_rev_per_student ====")
print(f"Test Accuracy: {lda_net['accuracy']:.4f}")
print(f"ROC AUC: {lda_net['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {lda_net['cv_scores'].mean():.4f} ± {lda_net['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(lda_net['class_report'])
print("\nTop 10 Important Features:")
print(lda_net['feature_importance'].head(10))

# Display results for deficit
print("\n==== LDA Results for deficit prediction ====")
print(f"Test Accuracy: {lda_deficit['accuracy']:.4f}")
print(f"ROC AUC: {lda_deficit['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {lda_deficit['cv_scores'].mean():.4f} ± {lda_deficit['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(lda_deficit['class_report'])
print("\nTop 10 Important Features:")
print(lda_deficit['feature_importance'].head(10))

# Visualize confusion matrices
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
sns.heatmap(lda_perc['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('LDA Confusion Matrix - perc_capacity')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 3, 2)
sns.heatmap(lda_net['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('LDA Confusion Matrix - net_rev_per_student')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 3, 3)
sns.heatmap(lda_deficit['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('LDA Confusion Matrix - deficit')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.savefig('lda_confusion_matrices.png')
plt.show()

# Visualize ROC curves
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.plot(lda_perc['fpr'], lda_perc['tpr'], label=f'LDA (AUC = {lda_perc["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - perc_capacity')
plt.legend(loc='lower right')

plt.subplot(1, 3, 2)
plt.plot(lda_net['fpr'], lda_net['tpr'], label=f'LDA (AUC = {lda_net["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - net_rev_per_student')
plt.legend(loc='lower right')

plt.subplot(1, 3, 3)
plt.plot(lda_deficit['fpr'], lda_deficit['tpr'], label=f'LDA (AUC = {lda_deficit["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - deficit')
plt.legend(loc='lower right')

plt.tight_layout()
plt.savefig('lda_roc_curves.png')
plt.show()

# Visualize feature importance
plt.figure(figsize=(18, 15))
plt.subplot(3, 1, 1)
top10_perc = lda_perc['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_perc)
plt.title('Top 10 Features - perc_capacity LDA')
plt.xlabel('Importance')

plt.subplot(3, 1, 2)
top10_net = lda_net['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_net)
plt.title('Top 10 Features - net_rev_per_student LDA')
plt.xlabel('Importance')

plt.subplot(3, 1, 3)
top10_deficit = lda_deficit['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_deficit)
plt.title('Top 10 Features - deficit LDA')
plt.xlabel('Importance')

plt.tight_layout()
plt.savefig('lda_feature_importance.png')
plt.show()

# Compare model performance across targets
models = ['perc_capacity', 'net_rev_per_student', 'deficit']
accuracies = [
    lda_perc['accuracy'], 
    lda_net['accuracy'],
    lda_deficit['accuracy']
]
cv_accuracies = [
    lda_perc['cv_scores'].mean(),
    lda_net['cv_scores'].mean(),
    lda_deficit['cv_scores'].mean()
]
roc_aucs = [
    lda_perc['roc_auc'],
    lda_net['roc_auc'],
    lda_deficit['roc_auc']
]

# Create comprehensive performance dataframe
lda_performance = pd.DataFrame({
    'Target': models,
    'Test Accuracy': accuracies,
    'CV Accuracy': cv_accuracies,
    'ROC AUC': roc_aucs
})

print("\nLDA Performance Comparison:")
print(lda_performance.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

# Create bar chart of performance metrics
plt.figure(figsize=(12, 6))
x = np.arange(len(models))
width = 0.3

plt.bar(x - width, accuracies, width, label='Test Accuracy')
plt.bar(x, cv_accuracies, width, label='CV Accuracy')
plt.bar(x + width, roc_aucs, width, label='ROC AUC')

plt.xlabel('Target')
plt.ylabel('Score')
plt.title('LDA Model Performance Comparison')
plt.xticks(x, models)
plt.legend()
plt.tight_layout()
plt.savefig('lda_performance_comparison.png')
plt.show()

print("\nLDA classification analysis completed!")

In [ ]:
# Random Forest Classification Script
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Prepare binary target variables
# Calculate median of target variables to split into binary classes
perc_capacity_median = df_filtered['perc_capacity'].median()
net_rev_median = df_filtered['net_rev_per_student'].median()

# Create binary target variables
df_filtered['perc_capacity_binary'] = (df_filtered['perc_capacity'] > perc_capacity_median).astype(int)
df_filtered['net_rev_binary'] = (df_filtered['net_rev_per_student'] > net_rev_median).astype(int)
# Create deficit binary target variable
df_filtered['deficit_binary'] = (df_filtered['net_program_80'] < 0).astype(int)

print("Class distribution for binary target variables:")
print("\nperc_capacity_binary:")
print(df_filtered['perc_capacity_binary'].value_counts())

print("\nnet_rev_binary:")
print(df_filtered['net_rev_binary'].value_counts())

print("\ndeficit_binary:")
print(df_filtered['deficit_binary'].value_counts())
print(f"Percentage of schools with deficit: {df_filtered['deficit_binary'].mean() * 100:.2f}%")

# Define features for deficit prediction - exclude net_program_80 which is used to calculate the target
deficit_features = [f for f in selected_features_net if f != 'net_program_80']

# Function to build and evaluate Random Forest model with ROC AUC calculation
def build_rf_model(X, y, feature_names):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Create a pipeline with scaling and Random Forest
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Predictions
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    
    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    
    # Get Random Forest feature importance
    rf = pipeline.named_steps['rf']
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': rf.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    return {
        'pipeline': pipeline,
        'accuracy': accuracy,
        'class_report': class_report,
        'conf_matrix': conf_matrix,
        'cv_scores': cv_scores,
        'feature_importance': feature_importance,
        'roc_auc': roc_auc,
        'fpr': fpr,
        'tpr': tpr
    }

# Prepare data for models - filter features that exist in the dataset
valid_perc_features = [f for f in selected_features_perc if f in df_filtered.columns]
valid_net_features = [f for f in selected_features_net if f in df_filtered.columns]
valid_deficit_features = [f for f in deficit_features if f in df_filtered.columns]

print(f"\nValid features for perc_capacity: {len(valid_perc_features)}/{len(selected_features_perc)}")
print(f"Valid features for net_rev_per_student: {len(valid_net_features)}/{len(selected_features_net)}")
print(f"Valid features for deficit prediction: {len(valid_deficit_features)}/{len(deficit_features)}")

# Prepare data for perc_capacity model
X_perc = df_filtered[valid_perc_features].copy()
X_perc = X_perc.fillna(X_perc.mean())  # Fill missing values
y_perc = df_filtered['perc_capacity_binary']

# Prepare data for net_rev model
X_net = df_filtered[valid_net_features].copy()
X_net = X_net.fillna(X_net.mean())  # Fill missing values
y_net = df_filtered['net_rev_binary']

# Prepare data for deficit model
X_deficit = df_filtered[valid_deficit_features].copy()
X_deficit = X_deficit.fillna(X_deficit.mean())  # Fill missing values
y_deficit = df_filtered['deficit_binary']

# Build Random Forest models
print("\n==== Building Random Forest models ====")
rf_perc = build_rf_model(X_perc, y_perc, valid_perc_features)
rf_net = build_rf_model(X_net, y_net, valid_net_features)
rf_deficit = build_rf_model(X_deficit, y_deficit, valid_deficit_features)

# Display results for perc_capacity
print("\n==== Random Forest Results for perc_capacity ====")
print(f"Test Accuracy: {rf_perc['accuracy']:.4f}")
print(f"ROC AUC: {rf_perc['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {rf_perc['cv_scores'].mean():.4f} ± {rf_perc['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(rf_perc['class_report'])
print("\nTop 10 Important Features:")
print(rf_perc['feature_importance'].head(10))

# Display results for net_rev
print("\n==== Random Forest Results for net_rev_per_student ====")
print(f"Test Accuracy: {rf_net['accuracy']:.4f}")
print(f"ROC AUC: {rf_net['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {rf_net['cv_scores'].mean():.4f} ± {rf_net['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(rf_net['class_report'])
print("\nTop 10 Important Features:")
print(rf_net['feature_importance'].head(10))

# Display results for deficit
print("\n==== Random Forest Results for deficit prediction ====")
print(f"Test Accuracy: {rf_deficit['accuracy']:.4f}")
print(f"ROC AUC: {rf_deficit['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {rf_deficit['cv_scores'].mean():.4f} ± {rf_deficit['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(rf_deficit['class_report'])
print("\nTop 10 Important Features:")
print(rf_deficit['feature_importance'].head(10))

# Visualize confusion matrices
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
sns.heatmap(rf_perc['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('RF Confusion Matrix - perc_capacity')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 3, 2)
sns.heatmap(rf_net['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('RF Confusion Matrix - net_rev_per_student')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 3, 3)
sns.heatmap(rf_deficit['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('RF Confusion Matrix - deficit')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.savefig('rf_confusion_matrices.png')
plt.show()

# Visualize ROC curves
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.plot(rf_perc['fpr'], rf_perc['tpr'], label=f'RF (AUC = {rf_perc["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - perc_capacity')
plt.legend(loc='lower right')

plt.subplot(1, 3, 2)
plt.plot(rf_net['fpr'], rf_net['tpr'], label=f'RF (AUC = {rf_net["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - net_rev_per_student')
plt.legend(loc='lower right')

plt.subplot(1, 3, 3)
plt.plot(rf_deficit['fpr'], rf_deficit['tpr'], label=f'RF (AUC = {rf_deficit["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - deficit')
plt.legend(loc='lower right')

plt.tight_layout()
plt.savefig('rf_roc_curves.png')
plt.show()

# Visualize feature importance
plt.figure(figsize=(18, 15))
plt.subplot(3, 1, 1)
top10_perc = rf_perc['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_perc)
plt.title('Top 10 Features - perc_capacity RF')
plt.xlabel('Importance')

plt.subplot(3, 1, 2)
top10_net = rf_net['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_net)
plt.title('Top 10 Features - net_rev_per_student RF')
plt.xlabel('Importance')

plt.subplot(3, 1, 3)
top10_deficit = rf_deficit['feature_importance'].head(10)
sns.barplot(x='Importance', y='Feature', data=top10_deficit)
plt.title('Top 10 Features - deficit RF')
plt.xlabel('Importance')

plt.tight_layout()
plt.savefig('rf_feature_importance.png')
plt.show()

# Compare model performance across targets
models = ['perc_capacity', 'net_rev_per_student', 'deficit']
accuracies = [
    rf_perc['accuracy'], 
    rf_net['accuracy'],
    rf_deficit['accuracy']
]
cv_accuracies = [
    rf_perc['cv_scores'].mean(),
    rf_net['cv_scores'].mean(),
    rf_deficit['cv_scores'].mean()
]
roc_aucs = [
    rf_perc['roc_auc'],
    rf_net['roc_auc'],
    rf_deficit['roc_auc']
]

# Create comprehensive performance dataframe
rf_performance = pd.DataFrame({
    'Target': models,
    'Test Accuracy': accuracies,
    'CV Accuracy': cv_accuracies,
    'ROC AUC': roc_aucs
})

print("\nRandom Forest Performance Comparison:")
print(rf_performance.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

# Create bar chart of performance metrics
plt.figure(figsize=(12, 6))
x = np.arange(len(models))
width = 0.3

plt.bar(x - width, accuracies, width, label='Test Accuracy')
plt.bar(x, cv_accuracies, width, label='CV Accuracy')
plt.bar(x + width, roc_aucs, width, label='ROC AUC')

plt.xlabel('Target')
plt.ylabel('Score')
plt.title('Random Forest Model Performance Comparison')
plt.xticks(x, models)
plt.legend()
plt.tight_layout()
plt.savefig('rf_performance_comparison.png')
plt.show()

print("\nRandom Forest classification analysis completed!")

In [ ]:
# Support Vector Machine Classification Script
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Prepare binary target variables
# Calculate median of target variables to split into binary classes
perc_capacity_median = df_filtered['perc_capacity'].median()
net_rev_median = df_filtered['net_rev_per_student'].median()

# Create binary target variables
df_filtered['perc_capacity_binary'] = (df_filtered['perc_capacity'] > perc_capacity_median).astype(int)
df_filtered['net_rev_binary'] = (df_filtered['net_rev_per_student'] > net_rev_median).astype(int)
# Create deficit binary target variable
df_filtered['deficit_binary'] = (df_filtered['net_program_80'] < 0).astype(int)

print("Class distribution for binary target variables:")
print("\nperc_capacity_binary:")
print(df_filtered['perc_capacity_binary'].value_counts())

print("\nnet_rev_binary:")
print(df_filtered['net_rev_binary'].value_counts())

print("\ndeficit_binary:")
print(df_filtered['deficit_binary'].value_counts())
print(f"Percentage of schools with deficit: {df_filtered['deficit_binary'].mean() * 100:.2f}%")

# Define features for deficit prediction - exclude net_program_80 which is used to calculate the target
deficit_features = [f for f in selected_features_net if f != 'net_program_80']

# Function to build and evaluate SVM model with ROC AUC calculation
def build_svm_model(X, y, feature_names):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Create a pipeline with scaling and SVM
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(probability=True, random_state=42))
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Predictions
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    
    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    
    # SVM doesn't provide direct feature importance
    # We could use permutation importance but will leave it for simplicity
    
    return {
        'pipeline': pipeline,
        'accuracy': accuracy,
        'class_report': class_report,
        'conf_matrix': conf_matrix,
        'cv_scores': cv_scores,
        'roc_auc': roc_auc,
        'fpr': fpr,
        'tpr': tpr
    }

# Prepare data for models - filter features that exist in the dataset
valid_perc_features = [f for f in selected_features_perc if f in df_filtered.columns]
valid_net_features = [f for f in selected_features_net if f in df_filtered.columns]
valid_deficit_features = [f for f in deficit_features if f in df_filtered.columns]

print(f"\nValid features for perc_capacity: {len(valid_perc_features)}/{len(selected_features_perc)}")
print(f"Valid features for net_rev_per_student: {len(valid_net_features)}/{len(selected_features_net)}")
print(f"Valid features for deficit prediction: {len(valid_deficit_features)}/{len(deficit_features)}")

# Prepare data for perc_capacity model
X_perc = df_filtered[valid_perc_features].copy()
X_perc = X_perc.fillna(X_perc.mean())  # Fill missing values
y_perc = df_filtered['perc_capacity_binary']

# Prepare data for net_rev model
X_net = df_filtered[valid_net_features].copy()
X_net = X_net.fillna(X_net.mean())  # Fill missing values
y_net = df_filtered['net_rev_binary']

# Prepare data for deficit model
X_deficit = df_filtered[valid_deficit_features].copy()
X_deficit = X_deficit.fillna(X_deficit.mean())  # Fill missing values
y_deficit = df_filtered['deficit_binary']

# Build SVM models
print("\n==== Building SVM models ====")
svm_perc = build_svm_model(X_perc, y_perc, valid_perc_features)
svm_net = build_svm_model(X_net, y_net, valid_net_features)
svm_deficit = build_svm_model(X_deficit, y_deficit, valid_deficit_features)

# Display results for perc_capacity
print("\n==== SVM Results for perc_capacity ====")
print(f"Test Accuracy: {svm_perc['accuracy']:.4f}")
print(f"ROC AUC: {svm_perc['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {svm_perc['cv_scores'].mean():.4f} ± {svm_perc['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(svm_perc['class_report'])

# Display results for net_rev
print("\n==== SVM Results for net_rev_per_student ====")
print(f"Test Accuracy: {svm_net['accuracy']:.4f}")
print(f"ROC AUC: {svm_net['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {svm_net['cv_scores'].mean():.4f} ± {svm_net['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(svm_net['class_report'])

# Display results for deficit
print("\n==== SVM Results for deficit prediction ====")
print(f"Test Accuracy: {svm_deficit['accuracy']:.4f}")
print(f"ROC AUC: {svm_deficit['roc_auc']:.4f}")
print(f"Cross-Validation Accuracy: {svm_deficit['cv_scores'].mean():.4f} ± {svm_deficit['cv_scores'].std():.4f}")
print("\nClassification Report:")
print(svm_deficit['class_report'])

# Visualize confusion matrices
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
sns.heatmap(svm_perc['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('SVM Confusion Matrix - perc_capacity')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 3, 2)
sns.heatmap(svm_net['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('SVM Confusion Matrix - net_rev_per_student')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 3, 3)
sns.heatmap(svm_deficit['conf_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('SVM Confusion Matrix - deficit')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.savefig('svm_confusion_matrices.png')
plt.show()

# Visualize ROC curves
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.plot(svm_perc['fpr'], svm_perc['tpr'], label=f'SVM (AUC = {svm_perc["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - perc_capacity')
plt.legend(loc='lower right')

plt.subplot(1, 3, 2)
plt.plot(svm_net['fpr'], svm_net['tpr'], label=f'SVM (AUC = {svm_net["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - net_rev_per_student')
plt.legend(loc='lower right')

plt.subplot(1, 3, 3)
plt.plot(svm_deficit['fpr'], svm_deficit['tpr'], label=f'SVM (AUC = {svm_deficit["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - deficit')
plt.legend(loc='lower right')

plt.tight_layout()
plt.savefig('svm_roc_curves.png')
plt.show()

# Compare model performance across targets
models = ['perc_capacity', 'net_rev_per_student', 'deficit']
accuracies = [
    svm_perc['accuracy'], 
    svm_net['accuracy'],
    svm_deficit['accuracy']
]
cv_accuracies = [
    svm_perc['cv_scores'].mean(),
    svm_net['cv_scores'].mean(),
    svm_deficit['cv_scores'].mean()
]
roc_aucs = [
    svm_perc['roc_auc'],
    svm_net['roc_auc'],
    svm_deficit['roc_auc']
]

# Create comprehensive performance dataframe
svm_performance = pd.DataFrame({
    'Target': models,
    'Test Accuracy': accuracies,
    'CV Accuracy': cv_accuracies,
    'ROC AUC': roc_aucs
})

print("\nSVM Performance Comparison:")
print(svm_performance.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

# Create bar chart of performance metrics
plt.figure(figsize=(12, 6))
x = np.arange(len(models))
width = 0.3

plt.bar(x - width, accuracies, width, label='Test Accuracy')
plt.bar(x, cv_accuracies, width, label='CV Accuracy')
plt.bar(x + width, roc_aucs, width, label='ROC AUC')

plt.xlabel('Target')
plt.ylabel('Score')
plt.title('SVM Model Performance Comparison')
plt.xticks(x, models)
plt.legend()
plt.tight_layout()
plt.savefig('svm_performance_comparison.png')
plt.show()



In [ ]:
# Combined Model Comparison Script
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# This script assumes you have already run the LDA, RF, and SVM scripts
# and have the results stored in variables like lda_perc, rf_perc, svm_perc, etc.

# Create function to extract metrics from classification reports
def extract_metrics_from_report(report):
    lines = report.strip().split('\n')
    for line in lines:
        if ' 1 ' in line:  # Class 1 metrics
            parts = [p for p in line.split() if p]
            precision = float(parts[1])
            recall = float(parts[2])
            f1 = float(parts[3])
            return precision, recall, f1
    return None, None, None

# Function to create comparison dataframe for a specific target
def create_comparison_df(target_name, lda_results, rf_results, svm_results):
    # Extract precision, recall, and F1 for each model
    lda_precision, lda_recall, lda_f1 = extract_metrics_from_report(lda_results['class_report'])
    rf_precision, rf_recall, rf_f1 = extract_metrics_from_report(rf_results['class_report'])
    svm_precision, svm_recall, svm_f1 = extract_metrics_from_report(svm_results['class_report'])
    
    # Create comparison dataframe
    comparison_df = pd.DataFrame({
        'Algorithm': ['LDA', 'Random Forest', 'SVM'],
        'Test Accuracy': [lda_results['accuracy'], rf_results['accuracy'], svm_results['accuracy']],
        'CV Accuracy': [lda_results['cv_scores'].mean(), rf_results['cv_scores'].mean(), svm_results['cv_scores'].mean()],
        'ROC AUC': [lda_results['roc_auc'], rf_results['roc_auc'], svm_results['roc_auc']],
        'Precision': [lda_precision, rf_precision, svm_precision],
        'Recall': [lda_recall, rf_recall, svm_recall],
        'F1-Score': [lda_f1, rf_f1, svm_f1]
    })
    
    return comparison_df

# Create comparison dataframes for each target
perc_capacity_comparison = create_comparison_df(
    'perc_capacity', 
    lda_perc, 
    rf_perc, 
    svm_perc
)

net_rev_comparison = create_comparison_df(
    'net_rev_per_student', 
    lda_net, 
    rf_net, 
    svm_net
)

deficit_comparison = create_comparison_df(
    'deficit', 
    lda_deficit, 
    rf_deficit, 
    svm_deficit
)

# Display comparison results
print("\n==== Model Comparison for Percent Capacity ====")
print(perc_capacity_comparison.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

print("\n==== Model Comparison for Net Revenue ====")
print(net_rev_comparison.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

print("\n==== Model Comparison for Deficit ====")
print(deficit_comparison.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

# Visualize model comparisons

# Accuracy comparison
plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.barplot(x='Algorithm', y='Test Accuracy', data=perc_capacity_comparison)
plt.title('Test Accuracy - Percent Capacity')
plt.ylim(0.5, 1.0)

plt.subplot(1, 3, 2)
sns.barplot(x='Algorithm', y='Test Accuracy', data=net_rev_comparison)
plt.title('Test Accuracy - Net Revenue')
plt.ylim(0.5, 1.0)

plt.subplot(1, 3, 3)
sns.barplot(x='Algorithm', y='Test Accuracy', data=deficit_comparison)
plt.title('Test Accuracy - Deficit')
plt.ylim(0.5, 1.0)

plt.tight_layout()
plt.savefig('test_accuracy_comparison.png')
plt.show()

# ROC AUC comparison
plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.barplot(x='Algorithm', y='ROC AUC', data=perc_capacity_comparison)
plt.title('ROC AUC - Percent Capacity')
plt.ylim(0.5, 1.0)

plt.subplot(1, 3, 2)
sns.barplot(x='Algorithm', y='ROC AUC', data=net_rev_comparison)
plt.title('ROC AUC - Net Revenue')
plt.ylim(0.5, 1.0)

plt.subplot(1, 3, 3)
sns.barplot(x='Algorithm', y='ROC AUC', data=deficit_comparison)
plt.title('ROC AUC - Deficit')
plt.ylim(0.5, 1.0)

plt.tight_layout()
plt.savefig('roc_auc_comparison.png')
plt.show()

# F1 Score comparison
plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.barplot(x='Algorithm', y='F1-Score', data=perc_capacity_comparison)
plt.title('F1 Score - Percent Capacity')
plt.ylim(0.5, 1.0)

plt.subplot(1, 3, 2)
sns.barplot(x='Algorithm', y='F1-Score', data=net_rev_comparison)
plt.title('F1 Score - Net Revenue')
plt.ylim(0.5, 1.0)

plt.subplot(1, 3, 3)
sns.barplot(x='Algorithm', y='F1-Score', data=deficit_comparison)
plt.title('F1 Score - Deficit')
plt.ylim(0.5, 1.0)

plt.tight_layout()
plt.savefig('f1_score_comparison.png')
plt.show()

# Create radar charts for comprehensive model comparison
def create_radar_chart(target_comparison, title):
    # Extract metrics for radar chart
    labels = ['Accuracy', 'ROC AUC', 'Precision', 'Recall', 'F1-Score']
    
    # Get values for each algorithm
    lda_values = [
        target_comparison.loc[0, 'Test Accuracy'],
        target_comparison.loc[0, 'ROC AUC'],
        target_comparison.loc[0, 'Precision'],
        target_comparison.loc[0, 'Recall'],
        target_comparison.loc[0, 'F1-Score']
    ]
    
    rf_values = [
        target_comparison.loc[1, 'Test Accuracy'],
        target_comparison.loc[1, 'ROC AUC'],
        target_comparison.loc[1, 'Precision'],
        target_comparison.loc[1, 'Recall'],
        target_comparison.loc[1, 'F1-Score']
    ]
    
    svm_values = [
        target_comparison.loc[2, 'Test Accuracy'],
        target_comparison.loc[2, 'ROC AUC'],
        target_comparison.loc[2, 'Precision'],
        target_comparison.loc[2, 'Recall'],
        target_comparison.loc[2, 'F1-Score']
    ]
    
    # Set up the radar chart
    angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]  # Close the loop
    
    # Add the last value to close the loop for each algorithm
    lda_values += lda_values[:1]
    rf_values += rf_values[:1]
    svm_values += svm_values[:1]
    labels += labels[:1]
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    
    # Plot each algorithm
    ax.plot(angles, lda_values, 'o-', linewidth=2, label='LDA')
    ax.plot(angles, rf_values, 'o-', linewidth=2, label='Random Forest')
    ax.plot(angles, svm_values, 'o-', linewidth=2, label='SVM')
    
    # Fill area
    ax.fill(angles, lda_values, alpha=0.1)
    ax.fill(angles, rf_values, alpha=0.1)
    ax.fill(angles, svm_values, alpha=0.1)
    
    # Set labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels[:-1])
    
    # Set y-axis limits
    ax.set_ylim(0.5, 1.0)
    
    # Add legend and title
    plt.legend(loc='upper right')
    plt.title(title)
    
    return fig

# Create radar charts for each target
perc_radar = create_radar_chart(perc_capacity_comparison, 'Model Comparison - Percent Capacity')
perc_radar.savefig('perc_capacity_radar.png')
plt.close(perc_radar)

net_rev_radar = create_radar_chart(net_rev_comparison, 'Model Comparison - Net Revenue')
net_rev_radar.savefig('net_rev_radar.png')
plt.close(net_rev_radar)

deficit_radar = create_radar_chart(deficit_comparison, 'Model Comparison - Deficit')
deficit_radar.savefig('deficit_radar.png')
plt.close(deficit_radar)

# Create a final combined ROC curve plot for all models and targets
plt.figure(figsize=(20, 6))

# Percent Capacity ROC curves
plt.subplot(1, 3, 1)
plt.plot(lda_perc['fpr'], lda_perc['tpr'], label=f'LDA (AUC = {lda_perc["roc_auc"]:.3f})')
plt.plot(rf_perc['fpr'], rf_perc['tpr'], label=f'RF (AUC = {rf_perc["roc_auc"]:.3f})')
plt.plot(svm_perc['fpr'], svm_perc['tpr'], label=f'SVM (AUC = {svm_perc["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Percent Capacity')
plt.legend(loc='lower right')

# Net Revenue ROC curves
plt.subplot(1, 3, 2)
plt.plot(lda_net['fpr'], lda_net['tpr'], label=f'LDA (AUC = {lda_net["roc_auc"]:.3f})')
plt.plot(rf_net['fpr'], rf_net['tpr'], label=f'RF (AUC = {rf_net["roc_auc"]:.3f})')
plt.plot(svm_net['fpr'], svm_net['tpr'], label=f'SVM (AUC = {svm_net["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Net Revenue')
plt.legend(loc='lower right')

# Deficit ROC curves
plt.subplot(1, 3, 3)
plt.plot(lda_deficit['fpr'], lda_deficit['tpr'], label=f'LDA (AUC = {lda_deficit["roc_auc"]:.3f})')
plt.plot(rf_deficit['fpr'], rf_deficit['tpr'], label=f'RF (AUC = {rf_deficit["roc_auc"]:.3f})')
plt.plot(svm_deficit['fpr'], svm_deficit['tpr'], label=f'SVM (AUC = {svm_deficit["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Deficit')
plt.legend(loc='lower right')

plt.tight_layout()
plt.savefig('combined_roc_curves.png')
plt.show()

# Identify best algorithm for each target based on ROC AUC
best_perc_algo = perc_capacity_comparison.loc[perc_capacity_comparison['ROC AUC'].idxmax(), 'Algorithm']
best_perc_auc = perc_capacity_comparison['ROC AUC'].max()
best_net_algo = net_rev_comparison.loc[net_rev_comparison['ROC AUC'].idxmax(), 'Algorithm']
best_net_auc = net_rev_comparison['ROC AUC'].max()
best_deficit_algo = deficit_comparison.loc[deficit_comparison['ROC AUC'].idxmax(), 'Algorithm']
best_deficit_auc = deficit_comparison['ROC AUC'].max()

print("\n==== Best Models Based on ROC AUC ====")
print(f"Percent Capacity: {best_perc_algo} (AUC = {best_perc_auc:.4f})")
print(f"Net Revenue: {best_net_algo} (AUC = {best_net_auc:.4f})")
print(f"Deficit: {best_deficit_algo} (AUC = {best_deficit_auc:.4f})")

# Create a final summary table
summary_df = pd.DataFrame({
    'Target': ['Percent Capacity', 'Net Revenue', 'Deficit'],
    'Best Algorithm': [best_perc_algo, best_net_algo, best_deficit_algo],
    'ROC AUC': [best_perc_auc, best_net_auc, best_deficit_auc]
})

print("\n==== Final Model Selection Summary ====")
print(summary_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

print("\nModel comparison analysis complete!")